In [3]:
import os
from pathlib import Path

In [ ]:
os.getcwd()

In [5]:
mlflow_logs = os.path.join(Path(os.getcwd()).parents[0], 'mlflow_logs')

In [7]:
os.listdir(mlflow_logs)

['.trash', '132899082267310288', '815316816171043374', 'models']

In [8]:
def is_experiment_folder(folder_name, base_path):
    folder_path = os.path.join(base_path, folder_name)
    if not os.path.isdir(folder_path):
        return False
    if folder_name == "models" or folder_name.startswith("."):
        return False
    meta_file = os.path.join(folder_path, "meta.yaml")
    return os.path.isfile(meta_file)

In [9]:
# Get list of experiments
experiment_folders = [
    folder for folder in os.listdir(mlflow_logs)
    if is_experiment_folder(folder, mlflow_logs)
]

print("Detected Experiment Folders:", experiment_folders)

Detected Experiment Folders: ['132899082267310288', '815316816171043374']


In [12]:
for folder in experiment_folders:
    exp_folder = os.path.join(mlflow_logs, folder) 
    print(exp_folder)
    print(os.listdir(exp_folder))

    print(5*'=')

/mnt/f/chatbot_ui_v2/mlflow_logs/132899082267310288
['meta.yaml']
=====
/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374
['meta.yaml', 'traces']
=====


In [13]:
traces_folder = os.path.join('/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374', 'traces')
os.listdir(traces_folder)

['05b2cdf0c02049478b1b37399282d88c',
 '1eed59cbad4d48308680c48d5ec0e1aa',
 '378981db883d47b0877df0fe4b7903c0',
 '52c0ffea5bb148efb18513566d3554c4',
 '5fc1a238ed0443efab9dc12653f10075',
 '6eac28386a9d433283b1fd138a7f6f59',
 '7603e45cc04a44c7b8531b0ad623f4be',
 '7afd6f476b4a410e97e7598f5529bb5d',
 '7b4609ab15004115bba9320920b87e38',
 '808dc0fa50214ae89aebb9bc5ea69f52',
 'a412f07738e049ec8ee5208549380bbc',
 'b00bdbdac88d4a4f814c3b410f572b5a',
 'b0bab827b3794d2a9a573cf8d3b58ad4',
 'b16d050a00c545528f3a1895c3817a06',
 'bde80301c45143119714869cdf543352',
 'd9f2a9609f13414782e0bf6de4ae367d',
 'da1705f1a66147199f81c460e2cfe939',
 'db2b69d101b34825b168e6fc905f9763',
 'dc93739ca3e044328461dc1fe054601a',
 'e0f93cd1e0174f0ba57eb1fe18af6998',
 'fe8877863c254926812aef905e0d972e']

In [15]:
import os
import yaml

def load_meta_yaml(file_path: str) -> dict:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"{file_path} does not exist.")
    
    with open(file_path, "r") as f:
        data = yaml.safe_load(f)
    
    return data

In [50]:
def get_readable_time(timestamp_ms ):
    timestamp_s = timestamp_ms / 1000
    readable_time = datetime.datetime.utcfromtimestamp(timestamp_s)
    return readable_time.strftime("%Y-%m-%d %H:%M:%S UTC")

In [119]:
class Experiment:
    def __init__(self, metadata):
        self.metadata = metadata
        self.artifact_location = self.metadata.get('artifact_location')
        self.creation_time = get_readable_time(self.metadata.get('creation_time'))
        self.experiment_id = self.metadata.get('experiment_id')
        self.name = self.metadata.get('name')
        self._trace_list = [] or self.get_trace_list(self.artifact_location)
        self.trace_list = [os.path.join(self.artifact_location, 'traces', i) for i in self._trace_list]                                     
        self.trace_list = [Trace(i) for i in self.trace_list]
    def __repr__(self):
        return str(self.metadata['name'])

    def get_experiment_details(self, experiment):
        file_to_open = os.path.join(self.ml_flow_logs_dir, experiment, 'meta.yaml')
        details = load_meta_yaml(file_to_open)
    
    def get_trace_list(self, exp_loc):
        trace_list = []
        traces_folder = os.path.join(exp_loc, 'traces')
        if os.path.exists(traces_folder):
            trace_list = os.listdir(traces_folder)
        return trace_list


class MLFLOW_LOGS:
    def __init__(self, ml_flow_logs_dir):
        self.ml_flow_logs_dir = ml_flow_logs_dir
        self.experiments = [] or self.get_experiments_done()
        self.models = []
    def get_experiments_done(self):
        exp_list = []
        for folder in os.listdir(self.ml_flow_logs_dir):
            if is_experiment_folder(folder, self.ml_flow_logs_dir):
                exp_list.append(folder)
        return exp_list

    def get_experiment_details(self, experiment):
        file_to_open = os.path.join(self.ml_flow_logs_dir, experiment, 'meta.yaml')
        details = load_meta_yaml(file_to_open)
        return details

    def get_experiment_list(self):
        exp_list = []
        for exp in self.experiments:
            details = self.get_experiment_details(exp)
            exp_list.append(Experiment(metadata = details))
        return exp_list         
    def __iter__(self):
        return iter(self.get_experiment_list())
mlflow_log_manager = MLFLOW_LOGS(ml_flow_logs_dir = mlflow_logs)
experiment_list = mlflow_log_manager.get_experiment_list()

In [130]:
for exp in mlflow_log_manager:
    print(exp)
    print(10*'=')
    for trace in exp.trace_list:
        print(trace.base_location)
        info = trace.get_span_data()
        #print(list(info.keys()))
        for i in info['spans']:
            #print(sorted(i['attributes'].keys()))
            #print(i['attributes']['mlflow.spanType'])
            print(i.get('name'))
        print(5*'-')

chatbot_debug_logs
agent testing
/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374/traces/05b2cdf0c02049478b1b37399282d88c
Workflow.run
BaseWorkflowAgent.init_run
BaseWorkflowAgent.setup_agent
BaseWorkflowAgent.run_agent_step
Ollama.astream_chat
ReActOutputParser.parse
BaseWorkflowAgent.parse_agent_output
Workflow._done
-----
/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374/traces/1eed59cbad4d48308680c48d5ec0e1aa
Workflow.run
BaseWorkflowAgent.init_run
BaseWorkflowAgent.setup_agent
BaseWorkflowAgent.run_agent_step
Ollama.astream_chat
ReActOutputParser.parse
BaseWorkflowAgent.parse_agent_output
Workflow._done
-----
/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374/traces/378981db883d47b0877df0fe4b7903c0
Workflow.run
BaseWorkflowAgent.init_run
BaseWorkflowAgent.setup_agent
BaseWorkflowAgent.run_agent_step
Ollama.astream_chat
ReActOutputParser.parse
BaseWorkflowAgent.parse_agent_output
Workflow._done
-----
/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374/traces/52c0ffea5bb148efb

In [80]:
#experiment_list

In [81]:
len(experiment_list)

2

In [82]:
for i in experiment_list:
    print(i)

chatbot_debug_logs
agent testing


In [83]:
import datetime

In [84]:
exp = experiment_list[1]

In [85]:
exp.metadata

{'artifact_location': '/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374',
 'creation_time': 1751092595089,
 'experiment_id': '815316816171043374',
 'last_update_time': 1751092595089,
 'lifecycle_stage': 'active',
 'name': 'agent testing'}

In [86]:
exp.trace_list

['05b2cdf0c02049478b1b37399282d88c',
 '1eed59cbad4d48308680c48d5ec0e1aa',
 '378981db883d47b0877df0fe4b7903c0',
 '52c0ffea5bb148efb18513566d3554c4',
 '5fc1a238ed0443efab9dc12653f10075',
 '6eac28386a9d433283b1fd138a7f6f59',
 '7603e45cc04a44c7b8531b0ad623f4be',
 '7afd6f476b4a410e97e7598f5529bb5d',
 '7b4609ab15004115bba9320920b87e38',
 '808dc0fa50214ae89aebb9bc5ea69f52',
 'a412f07738e049ec8ee5208549380bbc',
 'b00bdbdac88d4a4f814c3b410f572b5a',
 'b0bab827b3794d2a9a573cf8d3b58ad4',
 'b16d050a00c545528f3a1895c3817a06',
 'bde80301c45143119714869cdf543352',
 'd9f2a9609f13414782e0bf6de4ae367d',
 'da1705f1a66147199f81c460e2cfe939',
 'db2b69d101b34825b168e6fc905f9763',
 'dc93739ca3e044328461dc1fe054601a',
 'e0f93cd1e0174f0ba57eb1fe18af6998',
 'fe8877863c254926812aef905e0d972e']

In [75]:
def get_trace_list(exp_loc):
    trace_list = []
    traces_folder = os.path.join(exp_loc, 'traces')
    if os.path.exists(traces_folder):
        trace_list = os.listdir(traces_folder)
    return trace_list

In [76]:
trace_list = get_trace_list(exp.metadata['artifact_location'])

In [77]:
trace_list

['05b2cdf0c02049478b1b37399282d88c',
 '1eed59cbad4d48308680c48d5ec0e1aa',
 '378981db883d47b0877df0fe4b7903c0',
 '52c0ffea5bb148efb18513566d3554c4',
 '5fc1a238ed0443efab9dc12653f10075',
 '6eac28386a9d433283b1fd138a7f6f59',
 '7603e45cc04a44c7b8531b0ad623f4be',
 '7afd6f476b4a410e97e7598f5529bb5d',
 '7b4609ab15004115bba9320920b87e38',
 '808dc0fa50214ae89aebb9bc5ea69f52',
 'a412f07738e049ec8ee5208549380bbc',
 'b00bdbdac88d4a4f814c3b410f572b5a',
 'b0bab827b3794d2a9a573cf8d3b58ad4',
 'b16d050a00c545528f3a1895c3817a06',
 'bde80301c45143119714869cdf543352',
 'd9f2a9609f13414782e0bf6de4ae367d',
 'da1705f1a66147199f81c460e2cfe939',
 'db2b69d101b34825b168e6fc905f9763',
 'dc93739ca3e044328461dc1fe054601a',
 'e0f93cd1e0174f0ba57eb1fe18af6998',
 'fe8877863c254926812aef905e0d972e']

In [87]:
def load_yaml(file_path: str) -> dict:
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"{file_path} does not exist.")
    
    with open(file_path, "r") as f:
        data = yaml.safe_load(f)
    
    return data

In [108]:
class Trace:
    def __init__(self, base_location):
        self.base_location = base_location
        self.info = load_yaml(os.path.join(self.base_location, 'trace_info.yaml'))
    def get_span_data(self):
        span_file = os.path.join(base_location, 'artifacts', 'traces.json')
        data = load_yaml(span_file)
        return data

In [109]:
base_location = '/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374/traces/1eed59cbad4d48308680c48d5ec0e1aa'

In [110]:
trace = Trace(base_location)

In [111]:
trace.info

{'assessments': [],
 'execution_time_ms': 29841,
 'experiment_id': '815316816171043374',
 'request_id': '1eed59cbad4d48308680c48d5ec0e1aa',
 'status': 'OK',
 'timestamp_ms': 1751206432209}

In [112]:
trace.get_span_data()

{'spans': [{'trace_id': 'XLsilX+rGDPWGhmzSmvUxw==',
   'span_id': 'urDWqmGwWNM=',
   'trace_state': '',
   'parent_span_id': '',
   'name': 'Workflow.run',
   'start_time_unix_nano': 1751206432254542264,
   'end_time_unix_nano': 1751206462051142150,
   'attributes': {'mlflow.spanType': '"CHAIN"',
    'mlflow.spanInputs': '{"ctx": null, "stepwise": false, "checkpoint_callback": null, "start_event": {"user_msg": "What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?", "chat_history": null}}',
    'mlflow.spanOutputs': '{"response": {"role": "assistant", "additional_kwargs": {"tool_calls": [], "thinking": "Okay, let\'s tackle this question step by step. The user is asking about the government position held by the woman who portrayed Corliss Archer in the film \\"Kiss and Tell.\\" \\n\\nFirst, I need to identify who the actress is. The film \\"Kiss and Tell\\" is a 2000 movie, and Corliss Archer is a character in it. I should find out which 

In [95]:
full_traceback_file = os.path.join(base_location, 'artifacts', 'traces.json')

data = load_yaml(full_traceback_file)

In [97]:
data.keys()

dict_keys(['spans'])

In [133]:
pwd

'/mnt/f/chatbot_ui_v2/notebooks'

In [134]:
import mlflow
mlflow.set_tracking_uri(mlflow_logs)
experiment_name = "agent testing"
experiment = mlflow.get_experiment_by_name(experiment_name)

In [135]:
experiment

<Experiment: artifact_location='/mnt/f/chatbot_ui_v2/mlflow_logs/815316816171043374', creation_time=1751092595089, experiment_id='815316816171043374', last_update_time=1751092595089, lifecycle_stage='active', name='agent testing', tags={}>

In [137]:
if experiment:
    experiment_id = experiment.experiment_id
    runs_df = mlflow.search_runs(experiment_ids=[experiment_id], order_by=["start_time DESC"])

In [138]:
runs_df

,run_id,experiment_id,status,artifact_uri,start_time,end_time


In [136]:
if experiment:
    experiment_id = experiment.experiment_id
    runs_df = mlflow.search_runs(experiment_ids=[experiment_id], order_by=["start_time DESC"])

    if not runs_df.empty:
        latest_run_id = runs_df.iloc[0]["run_id"]
        print(f"Latest Run ID: {latest_run_id}")